In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [33]:
s = Service("D:/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
questions = [] # questions array/list storing entire data ranging from question type,difficulty, question url etc
html_code = [] # whole html code array/list

In [ ]:
driver = webdriver.Chrome(service=s)
driver.get('https://leetcode.com/problemset/')

# Clicking settings button
settings_button = driver.find_element(By.XPATH, value='//*[@id="headlessui-popover-button-:r7:"]') 
settings_button.click()

# Wait for the option to appear and then click it
option_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, "//*[starts-with(@id, 'headlessui-switch-:')]"))
)
option_button.click()

for i in range(1,67): # for page 1 to page 67
    time.sleep(1)
    # Find the question divs using css selector
    elements = driver.find_elements(By.CSS_SELECTOR, "[role='row'].odd\\:bg-layer-1.even\\:bg-overlay-1.dark\\:odd\\:bg-dark-layer-bg.dark\\:even\\:bg-dark-fill-4")
    for element in elements:
        try:
            #extracting the html code
            html_code.append(element.get_attribute('outerHTML'))
            # extracting all the values of the question
            questions.append(element.text)
        except Exception as e:
            pass
    
    # click the next page button
    driver.find_element(By.XPATH, value='//*[@id="__next"]/div[1]/div[4]/div[2]/div[1]/div[4]/div[3]/nav/button[10]').click()

driver.quit()


In [40]:
question_no = []
question = []
topic_tags = []
Acceptance = []
Difficulty = []
for i in questions:
    temp = i.split('\n')
    question_no.append(int(temp[0].split('.')[0]))
    question.append(temp[0].split('.')[1])
    topic_tags.append(temp[1:-2])
    Acceptance.append(temp[-2])
    Difficulty.append(temp[-1])
    
print(len(questions))

4041


In [42]:
soup = BeautifulSoup(str(html_code),'html.parser')

In [43]:
premium = []
rows = soup.find_all('div',{'role':'row'})
for row in rows:
    if(row.find('svg', {'class':"ml-2 h-4 shrink-0"})):
        premium.append(True)
    else:
        premium.append(False)

print(len(premium), sum(premium))

4041 798


In [20]:
import pandas as pd

In [44]:
df = pd.DataFrame({'Question_No':question_no,'Question': question,'Topic_tags':topic_tags,'Acceptance_rate':Acceptance,'isPremium':premium,'Difficulty':Difficulty})
df

,Question_No,Question,Topic_tags,Acceptance_rate,isPremium,Difficulty
0,2490,Circular Sentence,[String],66.9%,False,Easy
1,1,Two Sum,"[Array, Hash Table]",54.1%,False,Easy
2,2,Add Two Numbers,"[Linked List, Math, Recursion]",44.5%,False,Medium
3,3,Longest Substring Without Repeating Characters,"[Hash Table, String, Sliding Window]",35.7%,False,Medium
4,4,Median of Two Sorted Arrays,"[Array, Binary Search, 1+]",41.8%,False,Hard
...,...,...,...,...,...,...
4036,3335,Total Characters in String After Transformati...,"[Hash Table, Math, String, 2+]",24.9%,False,Medium
4037,3336,Find the Number of Subsequences With Equal GCD,"[Array, Math, 2+]",38.0%,False,Hard
4038,3337,Total Characters in String After Transformati...,"[Hash Table, Math, String, 2+]",31.1%,False,Hard
4039,3338,Second Highest Salary II,[Database],96.1%,True,Medium


In [45]:
def generate_link(text,alt_text='description'):

    suffix = '-'.join(text.lower().strip().split(' '))
    return f'https://leetcode.com/problems/{suffix}/{alt_text}'

In [46]:
df['Question_Link'] = df['Question'].apply(generate_link)
df['Solution'] = df['Question'].apply(lambda x: generate_link(x,'solutions'))

In [47]:
df.head()

,Question_No,Question,Topic_tags,Acceptance_rate,isPremium,Difficulty,Question_Link,Solution
0,2490,Circular Sentence,[String],66.9%,False,Easy,https://leetcode.com/problems/circular-sentenc...,https://leetcode.com/problems/circular-sentenc...
1,1,Two Sum,"[Array, Hash Table]",54.1%,False,Easy,https://leetcode.com/problems/two-sum/description,https://leetcode.com/problems/two-sum/solutions
2,2,Add Two Numbers,"[Linked List, Math, Recursion]",44.5%,False,Medium,https://leetcode.com/problems/add-two-numbers/...,https://leetcode.com/problems/add-two-numbers/...
3,3,Longest Substring Without Repeating Characters,"[Hash Table, String, Sliding Window]",35.7%,False,Medium,https://leetcode.com/problems/longest-substrin...,https://leetcode.com/problems/longest-substrin...
4,4,Median of Two Sorted Arrays,"[Array, Binary Search, 1+]",41.8%,False,Hard,https://leetcode.com/problems/median-of-two-so...,https://leetcode.com/problems/median-of-two-so...


In [48]:
df.drop_duplicates(subset='Question_No',keep='first',inplace=True)
df.shape

(3306, 8)

In [49]:
df.sort_values('Question_No',inplace=True)

In [50]:
df.reset_index(drop=True,inplace=True)

In [52]:
df.to_csv('Leetcode_Questions_updated.csv',index=False)